In [5]:
import psycopg2
import numpy as np
import pandas as pd
from langchain_openai import OpenAIEmbeddings

## Query

In [6]:
# 결과 조회용 함수
def show(rows):
    for row in rows:
        print(row)

In [8]:
# OpenAI 임베딩 사용
import os
os.environ['OPENAI_API_KEY'] = open('API_KEY', 'r').read()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 검색
test_words = "cyberpunk|dystopia|paraplegic|marine|warrior"  # 검색할 문자열
k = 5  # 상위 5개 유사 벡터 검색

# 검색할 문자열의 임베딩
test_vec = embeddings.embed_query(test_words)
emb_dim = len(test_vec)

In [48]:
# PG에 연결
conn_config = {'host': 'localhost', 'dbname': 'my_vec_db', 'user': 'jaesolshin', 'port': 5432}
conn = psycopg2.connect(**conn_config)
cursor = conn.cursor()

# 유사도 기반 검색
# <-> : L2거리
cursor.execute("""
    SELECT id,
        keywords,
        embedding <-> '{}' AS similarity
    FROM imdb
    ORDER BY similarity
    LIMIT 5
""".format(test_vec))

result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()

(626, 'christmas|magic|santa claus|son|toy', 0.8710372632789818)
(405, 'christmas|christmas eve|elf|high tech|resentment', 0.8716797845982714)
(89, 'boy|christmas|christmas eve|north pole|train', 0.9256088482608571)
(906, 'act of kindness|christmas|christmas eve|neighbor|vacation', 0.9353248476079452)
(383, 'christmas|island|love|survival|talking to inanimate object', 0.9784875435521179)


## Creating Index

### HNSW L2 Index

In [32]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    CREATE INDEX IF NOT EXISTS imdb_hnsw_index_l2 ON imdb USING hnsw (embedding vector_l2_ops) WITH (m = 64, ef_construction = 256);
""".format(test_vec))

# PG에 커밋
conn.commit()
conn.close()

In [33]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    SET LOCAL enable_indexscan = ON;           
    EXPLAIN ANALYZE SELECT id,
        keywords,
        embedding <-> '{}' AS similarity
    FROM imdb
    ORDER BY similarity
    LIMIT 5
""".format(test_vec))


result = cursor.fetchall()
show(result)

# cursor.execute("DROP INDEX IF EXISTS imdb_hnsw_index_l2;")

# PG에 커밋
conn.commit()
conn.close()

('Limit  (cost=155.40..155.41 rows=5 width=63) (actual time=6.163..6.165 rows=5 loops=1)',)
('  ->  Sort  (cost=155.40..157.93 rows=1010 width=63) (actual time=6.162..6.163 rows=5 loops=1)',)
("        Sort Key: ((embedding <-> '[0.038481392,0.036943275,-0.003692191,0.014768764,0.034693066,-0.05696727,-0.008160561,-0.0068218308,0.03440823,0.013422913,-0.0029160122,-0.024168365,0.011436179,-0.0054724184,0.0060527725,-0.0023142954,-0.03212954,-0.03973467,-0.013216406,0.012155391,0.002987221,0.011350729,-0.00313498,-0.009036433,-0.026703408,0.01864254,0.004614349,0.024866214,0.03566151,0.037797783,0.004090962,-0.03588938,-0.012475832,-0.007804516,0.017973173,-0.044377502,-0.005102131,0.026817342,0.01606477,0.011649807,0.02774306,-0.07098122,0.05218202,0.08055172,-0.024481684,-0.013301858,0.01803014,-0.006519192,-0.0015888527,0.048479147,-0.050529968,0.011849192,0.032243475,0.11273823,-0.042440616,0.018955858,-0.05448919,0.0019849532,0.024325024,-0.028925132,0.034607615,0.01613598,0.025962

### HNSW Cosine Index

In [34]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    CREATE INDEX imdb_hnsw_index_cos ON imdb USING hnsw (embedding vector_cosine_ops) WITH (m = 64, ef_construction = 256);
""".format(test_vec))

# PG에 커밋
conn.commit()
conn.close()

In [35]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    SET hnsw.ef_search = 100;
    EXPLAIN ANALYZE SELECT id,
        keywords,
        embedding <=> '{}' AS similarity
    FROM imdb
    ORDER BY similarity
    LIMIT 5;
""".format(test_vec))
result = cursor.fetchall()
show(result)

cursor.execute("DROP INDEX IF EXISTS imdb_hnsw_index_cos;")

# PG에 커밋
conn.commit()
conn.close()

('Limit  (cost=155.40..155.41 rows=5 width=63) (actual time=5.998..5.999 rows=5 loops=1)',)
('  ->  Sort  (cost=155.40..157.93 rows=1010 width=63) (actual time=5.997..5.997 rows=5 loops=1)',)
("        Sort Key: ((embedding <=> '[0.038481392,0.036943275,-0.003692191,0.014768764,0.034693066,-0.05696727,-0.008160561,-0.0068218308,0.03440823,0.013422913,-0.0029160122,-0.024168365,0.011436179,-0.0054724184,0.0060527725,-0.0023142954,-0.03212954,-0.03973467,-0.013216406,0.012155391,0.002987221,0.011350729,-0.00313498,-0.009036433,-0.026703408,0.01864254,0.004614349,0.024866214,0.03566151,0.037797783,0.004090962,-0.03588938,-0.012475832,-0.007804516,0.017973173,-0.044377502,-0.005102131,0.026817342,0.01606477,0.011649807,0.02774306,-0.07098122,0.05218202,0.08055172,-0.024481684,-0.013301858,0.01803014,-0.006519192,-0.0015888527,0.048479147,-0.050529968,0.011849192,0.032243475,0.11273823,-0.042440616,0.018955858,-0.05448919,0.0019849532,0.024325024,-0.028925132,0.034607615,0.01613598,0.025962

### HNSW IVFFlat Index

In [36]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    CREATE INDEX imdb_ivf_index_l2 ON imdb USING ivfflat (embedding vector_l2_ops) WITH (lists = 100);
""".format(test_vec))

# PG에 커밋
conn.commit()
conn.close()

In [37]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도를 포함한 쿼리 실행
cursor.execute("""
    SET ivfflat.probes = 10;
    EXPLAIN ANALYZE SELECT id,
        keywords,
        embedding <-> '{}' AS similarity
    FROM imdb
    ORDER BY similarity
    LIMIT 5;
""".format(test_vec))

result = cursor.fetchall()
show(result)

cursor.execute("DROP INDEX IF EXISTS imdb_ivf_index_l2;")

# PG에 커밋
conn.commit()
conn.close()

('Limit  (cost=155.40..155.41 rows=5 width=63) (actual time=5.616..5.617 rows=5 loops=1)',)
('  ->  Sort  (cost=155.40..157.93 rows=1010 width=63) (actual time=5.615..5.615 rows=5 loops=1)',)
("        Sort Key: ((embedding <-> '[0.038481392,0.036943275,-0.003692191,0.014768764,0.034693066,-0.05696727,-0.008160561,-0.0068218308,0.03440823,0.013422913,-0.0029160122,-0.024168365,0.011436179,-0.0054724184,0.0060527725,-0.0023142954,-0.03212954,-0.03973467,-0.013216406,0.012155391,0.002987221,0.011350729,-0.00313498,-0.009036433,-0.026703408,0.01864254,0.004614349,0.024866214,0.03566151,0.037797783,0.004090962,-0.03588938,-0.012475832,-0.007804516,0.017973173,-0.044377502,-0.005102131,0.026817342,0.01606477,0.011649807,0.02774306,-0.07098122,0.05218202,0.08055172,-0.024481684,-0.013301858,0.01803014,-0.006519192,-0.0015888527,0.048479147,-0.050529968,0.011849192,0.032243475,0.11273823,-0.042440616,0.018955858,-0.05448919,0.0019849532,0.024325024,-0.028925132,0.034607615,0.01613598,0.025962

### Expression Indexing

Expression Indexing은 데이터베이스에서 특정 컬럼의 값뿐만 아니라, 해당 컬럼에 어떤 표현식(expression)을 적용한 결과에 대해 인덱스를 생성하는 방법을 의미함. \
여기서 "표현식"은 함수 호출, 산술 연산, 텍스트 변환 등과 같이 데이터를 변형하거나 계산하는 논리를 포함

#### Binary Hamming Distance Index + Rerank

In [38]:
len(test_vec)

1536

In [39]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# embedding을 bit타입으로 리코딩 후 hamming distance 기반의 HNSW 인덱스 생성
cursor.execute("""
    CREATE INDEX IF NOT EXISTS imdb_binary_index ON imdb USING hnsw ((binary_quantize(embedding)::bit(1536)) bit_hamming_ops);
""".format(test_vec))

# PG에 커밋
conn.commit()
conn.close()

In [40]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도 기반 쿼리 실행
# <~> : hamming distance
cursor.execute("""
    SELECT id,
        keywords,
        binary_quantize(embedding::vector) <~> binary_quantize('{}'::vector) AS hamming_distance
    FROM imdb
    ORDER BY hamming_distance
    LIMIT 5;
""".format(test_vec))

result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()

(1, 'avatar|future|marine|native|paraplegic', 403.0)
(1001, 'avatar|future|marine|native|paraplegic', 403.0)
(105, 'alternate timeline|cyborg|future|robot|time machine', 474.0)
(303, 'alien|cyborg|pirate|planet|treasure', 479.0)
(663, 'betrayal|futuristic|gladiator|robot|scientist', 494.0)


In [41]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# 유사도 기반 쿼리 실행 후, 원래의 값으로 cosine 유사도 기반 재정렬
cursor.execute("""
    SELECT id,
        keywords,
        embedding <=> '{}' AS similarity
    FROM(
        SELECT *,
            binary_quantize(embedding::vector) <~> binary_quantize('{}'::vector) AS hamming_distance
        FROM imdb
        ORDER BY hamming_distance
        LIMIT 5
    )
    ORDER BY similarity;
""".format(test_vec, test_vec))

result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()

(1, 'avatar|future|marine|native|paraplegic', 0.3071046518496289)
(1001, 'avatar|future|marine|native|paraplegic', 0.3071046518496289)
(303, 'alien|cyborg|pirate|planet|treasure', 0.4324717690804051)
(105, 'alternate timeline|cyborg|future|robot|time machine', 0.439113113933019)
(663, 'betrayal|futuristic|gladiator|robot|scientist', 0.4481391043580103)


#### Subvector Index + Rerank

In [42]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# embedding 중 1~100차원의 subvector 사용하여 cosine 거리 기반 HNSW 인덱스 생성
cursor.execute("""
CREATE INDEX IF NOT EXISTS imdb_subvector_index 
ON imdb 
USING hnsw ((subvector(embedding, 1, 100)::vector(100)) vector_cosine_ops);
""")

# subvector index를 바탕으로 10개의 후보 선정
cursor.execute("""
SELECT id,
   keywords,
   subvector(embedding, 1, 100)::vector(100) <=> subvector('{}'::vector, 1, 100)::vector(100) AS similarity
FROM imdb
ORDER BY similarity
LIMIT 10;
""".format(test_vec))

result = cursor.fetchall()
print('\nBy Subvector Indexing:')
show(result)

# 선정된 후보에 대해, 원래 벡터를 기반으로 재정렬
cursor.execute("""
SELECT id,
      keywords,
      embedding <=> '{}'::vector AS similarity
FROM (
   SELECT *,
          subvector(embedding, 1, 100)::vector(100) <=> subvector('{}'::vector, 1, 100)::vector(100) AS similarity
   FROM imdb
   ORDER BY similarity
   LIMIT 10
) subquery
ORDER BY similarity
LIMIT 5;
""".format(test_vec, test_vec))

print('\nAfter Reranking:')
result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()


By Subvector Indexing:
(1, 'avatar|future|marine|native|paraplegic', 0.30812167194748197)
(1001, 'avatar|future|marine|native|paraplegic', 0.30812167194748197)
(480, 'astronaut|mars|robot|robot as menace|survival', 0.3532452784042954)
(838, 'based on cult favorite|disease|female hero|female protagonist|future', 0.3552280774038875)
(263, 'humanoid robot|man versus machine|prosthetic limb|robot as pathos|robot vs. robot', 0.3616564649361832)
(773, 'based on comic book|breaking the fourth wall|mercenary|mutant|self healing', 0.3697674836286392)
(87, 'high tech|martial arts|masked man|robot|superhero', 0.37145353804428105)
(413, 'brutality|dark humor|female antagonist|female warrior|post apocalypse', 0.3801884341222339)
(662, 'fem bot|prison|seductive fembot|spy|time travel', 0.3916574096257399)
(911, 'dystopia|freedom|government|revolution|terrorist', 0.39359218031466914)

After Reranking:
(1, 'avatar|future|marine|native|paraplegic', 0.3071046518496289)
(1001, 'avatar|future|marine|nati

#### Full-Text Search + Reciprocal Rank Fusion

In [43]:
# PG에 연결
conn = psycopg2.connect(host='192.168.0.47', dbname='postgres', user='postgres', password='postgres1016', port=55432)
cursor = conn.cursor()

# Full-Text Search를 위한 GIN Index 생성
# GIN Index 생성을 위해서는 tsvector 타입의 컬럼이 필요하다
cursor.execute("""
-- tsvector 형식의 keyword 열 생성
ALTER TABLE imdb ADD COLUMN IF NOT EXISTS textsearch tsvector;
-- keyword열에 to_tsvector 함수를 적용하여 textsearch 열에 저장
-- 'english': 영어에 맞는 stopwords, stemming 설정 적용
UPDATE imdb SET textsearch = to_tsvector('english', keywords); 
-- textsearch에 기반하여 인덱스 생성. GIN 또는 GiST 인덱스를 생성할 수 있음.
CREATE INDEX IF NOT EXISTS textsearch_idx ON imdb USING GIN (textsearch);
""")

# PostgreSQL Full-Text Search
# 'Christmas'라는 문자열에 대해 검색
cursor.execute("""
SELECT id, keywords
FROM imdb
WHERE textsearch @@ plainto_tsquery('christmas')
ORDER BY ts_rank_cd(textsearch, plainto_tsquery('christmas')) DESC
LIMIT 5;
""")

print('\nFull Text-Search:')
result = cursor.fetchall()
show(result)


# Cosine 유사도 기반 벡터 검색
test_vec = embeddings.embed_query("christmas") # "Christmas"의 임베딩

cursor.execute("""
    SET hnsw.ef_search = 100;
    SELECT id,
        keywords,
        embedding <=> '{}' AS similarity
    FROM imdb
    ORDER BY similarity
    LIMIT 5;
""".format(test_vec))

print('\nVector Search:')
result = cursor.fetchall()
show(result)


# Full-Text Search 결과에 대해 Cosine 유사도 기반 재정렬
cursor.execute("""
SELECT id,
    keywords,
    embedding <=> '{}' AS similarity
FROM (
    SELECT *
    FROM imdb
    WHERE textsearch @@ plainto_tsquery('christmas')
    ORDER BY ts_rank_cd(textsearch, plainto_tsquery('christmas')) DESC
    LIMIT 50
)
ORDER BY similarity
LIMIT 5
""".format(test_vec))

print('\nReranked by Cosine Similarity after 50 Full Text-Search Results:')
result = cursor.fetchall()
show(result)

# Full-Text Search 결과와 Cosine 유사도 기반 결과에 대해
# Reciprocal Rank Fusion을 수행한 결과
cursor.execute(
"""
WITH fts_results AS (
    SELECT id,
           keywords,
           ROW_NUMBER() OVER (ORDER BY ts_rank_cd(textsearch, plainto_tsquery('christmas')) DESC) AS rank
    FROM imdb
    WHERE textsearch @@ plainto_tsquery('christmas')
    LIMIT 5
),
fts_rrf AS (
    SELECT id,
           keywords,
           1.0 / (60 + rank) AS rrf_score
    FROM fts_results
),
vector_results AS (
    SELECT id,
           keywords,
           ROW_NUMBER() OVER (ORDER BY embedding <=> '{}'::vector) AS rank
    FROM imdb
    LIMIT 5
),
vector_rrf AS (
    SELECT id,
           keywords,
           1.0 / (60 + rank) AS rrf_score
    FROM vector_results
),
combined_results AS (
    SELECT id, keywords, SUM(rrf_score) AS total_rrf_score
    FROM (
        SELECT * FROM fts_rrf
        UNION ALL
        SELECT * FROM vector_rrf
    ) subquery
    GROUP BY id, keywords
)
SELECT id, keywords, total_rrf_score
FROM combined_results
ORDER BY total_rrf_score DESC
LIMIT 5;
""".format(test_vec))

print('\nReciprocal Rank Fusion of Vector Search and Full-Text Search Results:')
result = cursor.fetchall()
show(result)

# PG에 커밋
conn.commit()
conn.close()


Full Text-Search:
(405, 'christmas|christmas eve|elf|high tech|resentment')
(60, 'charles dickens|christmas|christmas eve|ghost|scrooge')
(379, 'box office flop|christmas|christmas eve|critically bashed|toy comes to life')
(89, 'boy|christmas|christmas eve|north pole|train')
(906, 'act of kindness|christmas|christmas eve|neighbor|vacation')

Vector Search:
(626, 'christmas|magic|santa claus|son|toy', 0.3793529272079468)
(405, 'christmas|christmas eve|elf|high tech|resentment', 0.37991273403167725)
(89, 'boy|christmas|christmas eve|north pole|train', 0.42837557623531863)
(906, 'act of kindness|christmas|christmas eve|neighbor|vacation', 0.43741653673102043)
(383, 'christmas|island|love|survival|talking to inanimate object', 0.47871855028161014)

Reranked by Cosine Similarity after 50 Full Text-Search Results:
(626, 'christmas|magic|santa claus|son|toy', 0.3793529272079468)
(405, 'christmas|christmas eve|elf|high tech|resentment', 0.37991273403167725)
(89, 'boy|christmas|christmas eve|n